In [4]:
import pandas as pd
from datetime import datetime


#prepare data before importing to database
chunksize = 100000
chunks = pd.read_csv("data/Google-Playstore.csv",
                     chunksize=chunksize
                     )

def convert_size(value):
    value = value.replace(",", "")

    if "G" in value:
        return float(value.replace("G", "")) * 1_000_000_000
    elif "M" in value:
        return float(value.replace("M", "")) * 1_000_000  # Convert M to numeric
    elif "k" in value:
        return float(value.replace("k", "")) * 1_000  # Convert K to numeric
    elif "Varies with device" in value:
        return None  # Handle special case
    else:
        return float(value)  # Already numeric

def convert_dates(date):
    if date != "nan":
        date_obj = datetime.strptime(date, "%b %d, %Y")
        return date_obj.strftime("%d-%m-%Y")
    else:
        return None

columns_to_clean = ["Free", "Ad Supported", "In App Purchases", "Editors Choice"]

df_list = []
for chunk in chunks:
    chunk.dropna(how='all', inplace=True)
    bool_columns = ["Free", "Ad Supported", "In App Purchases", "Editors Choice"]
    chunk[bool_columns] = chunk[bool_columns].astype(bool).replace({True: 1, False: 0}).astype(str)
    chunk["Installs"] = chunk["Installs"].astype(str).replace(r"[+,]", "", regex=True).astype(str)
    chunk["Size"] = chunk["Size"].astype(str).apply(convert_size)
    chunk["Minimum Android"] = chunk["Minimum Android"].astype(str).replace(r"(W|and up)\b", "", regex=True).str.strip()
    chunk["Released"] = chunk["Released"].astype(str).apply(convert_dates)
    chunk["Last Updated"] = chunk["Last Updated"].astype(str).apply(convert_dates)
    df_list.append(chunk)


df = pd.concat(df_list, ignore_index=True)
df.drop("Scraped Time", axis=1, inplace=True)
df.to_csv("data/Google-Playstore_cleaned.csv", index=False)
df.head(50)

C:\Users\A\AppData\Local\Temp\ipykernel_9368\3692365626.py:47: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  chunk[bool_columns] = chunk[bool_columns].astype(bool).replace({True: 1, False: 0}).astype(str)
C:\Users\A\AppData\Local\Temp\ipykernel_9368\3692365626.py:47: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  chunk[bool_columns] = chunk[bool_columns].astype(bool).replace({True: 1, False: 0}).astype(str)
C:\Users\A\AppData\Local\Temp\ipykernel_9368\3692365626.py:47: FutureWarning: Downcasting behavior in `replace` is deprecated 

,App Name,App Id,Category,Rating,Rating Count,Installs,Minimum Installs,Maximum Installs,Free,Price,...,Developer Id,Developer Website,Developer Email,Released,Last Updated,Content Rating,Privacy Policy,Ad Supported,In App Purchases,Editors Choice
0,Gakondo,com.ishakwe.gakondo,Adventure,0.0,0.0,10,10.0,15,1,0.0,...,Jean Confident Irénée NIYIZIBYOSE,https://beniyizibyose.tk/#/,jean21101999@gmail.com,26-02-2020,26-02-2020,Everyone,https://beniyizibyose.tk/projects/,0,0,0
1,Ampere Battery Info,com.webserveis.batteryinfo,Tools,4.4,64.0,5000,5000.0,7662,1,0.0,...,Webserveis,https://webserveis.netlify.app/,webserveis@gmail.com,21-05-2020,06-05-2021,Everyone,https://dev4phones.wordpress.com/licencia-de-uso/,1,0,0
4,GROW.me,com.horodyski.grower,Tools,0.0,0.0,100,100.0,478,1,0.0,...,Rafal Milek-Horodyski,http://www.horodyski.com.pl,rmilekhorodyski@gmail.com,21-02-2020,12-11-2018,Everyone,http://www.horodyski.com.pl,0,0,0
5,IMOCCI,com.imocci,Social,0.0,0.0,50,50.0,89,1,0.0,...,Imocci GmbH,http://www.imocci.com,info@imocci.com,24-12-2018,20-12-2019,Teen,https://www.imocci.com/wp-content/uploads/2018...,0,1,0
9,Neon 3d Iron Tech Keyboard Theme,com.ikeyboard.theme.neon_3d.iron.tech,Personalization,4.7,820.0,50000,50000.0,62433,1,0.0,...,Free 2021 Themes for Emoji keyboard,https://trendyteme888-31139.web.app,trendyteme.888@gmail.com,22-09-2019,07-10-2020,Everyone,http://bit.ly/EmojiThemePro,1,0,0
11,Parents,com.eqra.android.BSTS.parent,Maps & Navigation,0.0,0.0,100,100.0,330,1,0.0,...,Eqra Tech,http://www.hafizquran.com,hhiyassat@eqratech.com,10-01-2018,27-06-2018,Everyone,https://docs.google.com/document/d/1fpoF-YmfcV...,0,0,0
12,桃園機場捷運時刻表 - 捷運轉乘路線快速查詢(支援台北捷運),com.jieapp.taoyuanmetro,Travel & Local,3.9,118.0,10000,10000.0,37763,1,0.0,...,Jie App,https://jie-app.herokuapp.com/,jie.myapp@gmail.com,03-04-2018,11-06-2021,Everyone,http://appsetting.blogspot.tw/2017/02/jie-app....,1,0,0
13,be.MOBILISED,com.hastobe.bemobilised,Maps & Navigation,0.0,0.0,100,100.0,274,1,0.0,...,has.to.be gmbh,http://www.has-to-be.com,support@has-to-be.com,09-02-2020,14-05-2021,Everyone,https://has-to-be.com/de/ueber-uns/datenschutz...,0,0,0
14,Caliway Conductor,com.caliwayconductor.driver,Travel & Local,3.7,1572.0,10000,10000.0,42856,1,0.0,...,WayCali,https://www.facebook.com/waycali,waycalicol@gmail.com,05-09-2018,30-05-2020,Everyone,https://www.facebook.com/waycali,0,0,0
15,Readymade Grocery App,com.ionicfirebaseapp.readymadegroceryuserapp,Food & Drink,4.2,16.0,1000,1000.0,4313,1,0.0,...,pietechsolution,https://www.ionicfirebaseapp.com/products/read...,info@ionicfirebaseapp.com,05-04-2020,23-03-2021,Everyone,https://www.ionicfirebaseapp.com/contact,0,0,0
